In [ ]:
# @title Environment Setup
import os, re, subprocess, sys
from pathlib import Path

def run(cmd):
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True)

try:
    run(['apt-get', 'update'])
    apt = Path('apt.txt')
    if apt.exists():
        pkgs = [p.strip() for p in apt.read_text().splitlines() if p.strip()]
        if pkgs:
            run(['apt-get', 'install', '-y', *pkgs])
except Exception as e:
    print('Skipping apt-get install:', e)

gpu = subprocess.run(['bash','-lc','nvidia-smi'], capture_output=True, text=True)
has_gpu = gpu.returncode == 0
index = os.environ.get('PYTORCH_INDEX_URL')
if not index and has_gpu:
    m = re.search(r'CUDA Version: (\d+)\.(\d+)', gpu.stdout or gpu.stderr)
    if m:
        index = f"https://download.pytorch.org/whl/cu{m.group(1)}{m.group(2)}"
if not index:
    index = "https://download.pytorch.org/whl/cpu"
print('Using PyTorch index:', index)
try:
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={index}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])
except subprocess.CalledProcessError:
    fallback = "https://download.pytorch.org/whl/cpu"
    print('PyTorch install failed, falling back to', fallback)
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={fallback}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])

req = 'requirements-colab-gpu.txt' if has_gpu else 'requirements-colab-cpu.txt'
run([sys.executable, '-m', 'pip', 'install', '-r', req])

print('Python', sys.version)
subprocess.run(['ffmpeg', '-version'])
import torch, torchvision, torchaudio, soxr
print('torch', torch.__version__)
print('torchvision', torchvision.__version__)
print('torchaudio', torchaudio.__version__)
print('CUDA available:', torch.cuda.is_available())
print('Resampler backend:', 'soxr', getattr(soxr, '__version__', 'unknown'))


In [ ]:
# @title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install helpers
!pip install --quiet gdown


In [ ]:
# Download and cache models (RVC/UVR)
import hashlib, os, shutil, subprocess

def md5(path):
    with open(path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

def run(cmd):
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True)

def download(url, dest):
    os.makedirs(os.path.dirname(dest), exist_ok=True)
    if url.startswith('gdrive:'):
        file_id = url.split(':', 1)[1]
        run(['gdown', '--id', file_id, '--output', dest])
    else:
        run(['wget','--retry-connrefused','--waitretry=5','--read-timeout=20',
             '--timeout=15','-t','0','-c', url, '-O', dest])

MODELS = {
    'rvc': {
        'url': '',  # TODO: set to model URL or gdrive:file_id
        'md5': None,
        'drive': '/content/drive/MyDrive/models/RVC/G_8200.pth',
        'local': '/content/models/RVC/G_8200.pth'
    },
    'uvr': {
        'url': '',
        'md5': None,
        'drive': '/content/drive/MyDrive/models/UVR/UVR_MDX.pth',
        'local': '/content/models/UVR/UVR_MDX.pth'
    }
}

def ensure_model(name):
    cfg = MODELS[name]
    local, drive, url, md5_hash = cfg['local'], cfg['drive'], cfg['url'], cfg.get('md5')
    if os.path.exists(local) and (not md5_hash or md5(local) == md5_hash):
        print('Using existing local model:', local)
        return local
    if os.path.exists(drive):
        print('Copying model from Drive:', drive)
        os.makedirs(os.path.dirname(local), exist_ok=True)
        shutil.copy2(drive, local)
        if md5_hash and md5(local) != md5_hash:
            raise ValueError('Checksum mismatch after copy')
        return local
    if not url:
        raise ValueError(f'No URL specified for {name}')
    print('Downloading model from', url)
    download(url, local)
    if md5_hash and md5(local) != md5_hash:
        raise ValueError('Checksum mismatch after download')
    os.makedirs(os.path.dirname(drive), exist_ok=True)
    shutil.copy2(local, drive)
    return local

for model_name in MODELS:
    try:
        ensure_model(model_name)
    except Exception as e:
        print(f'Skipping {model_name}:', e)


In [ ]:
# @title Upload stems to Drive
# Upload four WAV files named vocals.wav, drums.wav, bass.wav and other.wav
from google.colab import files
import os
INPUT_DIR = '/content/drive/MyDrive/AutoMix/input'  # @param {type:"string"}
os.makedirs(INPUT_DIR, exist_ok=True)
uploaded = files.upload()
for name, data in uploaded.items():
    with open(os.path.join(INPUT_DIR, name), 'wb') as f:
        f.write(data)
print('Uploaded files:', list(uploaded.keys()))


In [ ]:
# @title Run pipeline
import os, sys, subprocess
INPUT_DIR = '/content/drive/MyDrive/AutoMix/input'  # @param {type:"string"}
OUTPUT_DIR = '/content/drive/MyDrive/AutoMix/output'  # @param {type:"string"}
os.makedirs(OUTPUT_DIR, exist_ok=True)
cmd = [sys.executable, 'scripts/pipeline_gdrive.py', '--input', INPUT_DIR, '--output', OUTPUT_DIR]
subprocess.run(cmd, check=True)
print('Output files:', os.listdir(OUTPUT_DIR))


In [ ]:
# @title Play mixed result
import os
from IPython.display import Audio
OUTPUT_DIR = '/content/drive/MyDrive/AutoMix/output'  # @param {type:"string"}
audio_path = os.path.join(OUTPUT_DIR, 'mix.wav')
if os.path.exists(audio_path):
    display(Audio(filename=audio_path))
else:
    print('mix.wav not found')
